## 03 Reproduction Analysis with Self-developed Python Package, 'DIM' 

written by **Jinwoo Lee**            
jil527@ucsd.edu | jinwoo-lee.com

Nov 12th, 2025  
as a PSYC201A's **Reproduction Project**

**Note:** This script aims to reproduce the following result with the authors' original code and data.      
> Mantel tests revealed that the (young adults’) left amygdala–ventral PFC tract-morphology dissimilarity matrix was significantly correlated with the trait-anxiety dissimilarity matrix (*P* = 0.036).

For this purpose, we will utilize parts of the **‘DIM’** package, which was custom-developed for the First Year Project. Since **DIM** is part of an ongoing research project in our lab, the corresponding GitHub repository is not publicly available.

---
### Step 1. Loading the Packages

#### Overview of DIM Package
In addition to basic packages such as pandas, I will import my DIM package. `Level1` class is specifically designed for performing conventional inter-subject representational similarity analysis (IS-RSA; e.g., Anna Karenina Modeling).    

When running a traditional IS-RSA with a `Level1` model, the following parameters should be defined first:

- **model**: Choose one of the two theoretically defined IS-RSA models; ['AnnaK' or 'NearestN']  
  - `AnnaK`: assuming that participants with higher trait anxiety scores will show (dis)similar brain morphology.
  - `NearestN`: assuming that participants with (dis)similar triat anxiety scores will show (dis)similar brain morphology.      
  *Example:* Kim & Kim (2022) tested the model using Anna Karenina → `AnnaK`

- **distance**: Define which metric to calculate the relationship between participants in the test domain (in our case, trait anxiety). Available metrics depend on the model.  
  - if `model == 'AnnaK'`: ['Mean' or 'Min']  
  - if `model == 'NearestN'`: ['Euclidean' or 'Cosine']       
  *Example:* Kim & Kim (2022) used the mean metric with the AnnaK model → `Mean`

- **weighting**: When there are two or more input variables in the test domain, choose how to weight each variable when calculating participant relationships; ['None' or 'Learn']  
  - `None`: weighting all X features equally
  - `Learn`: optimizing the weight of each X feature to maximize the correlation between the test and target domains    
  *Example:* Kim & Kim (2022) used only one variable (STAI total score), so no weighting was applied → `None`

- **dependency**: Choose the metric to calculate the correlation between domains; ['Pearson' or 'Spearman']  
  - `Pearson`: Pearson’s r  
  - `Spearman`: Spearman’s rho       
  *Example:* Kim & Kim (2022) calculated the correlation between STAI and brain distance matrices using Spearman’s rho → `Spearman`

The defined Level1 model gets **(n_subjects * n_features)** shaped test domain dataframe and **(n_subjects * n_subjects)** shaped distance matrix of the target domain. In my case, I will input (119, 1) shaped STAI total score vector and (119, 119) shaped brain morphology distance matrix. After that, `DIM`'s Level1 model will compute the distance matrix of STAI scores based on the specified IS-RSA model and paramters, and then calculate the correlation between the two distance matrices.

Please see 'Step 4' for the usage of `Level1` class. 

In [1]:
import sys
import pandas as pd

sys.path.insert(0, '../DIM/src')
from lib.dim.level1 import Level1  # importing our DIM package
from scipy.spatial.distance import pdist, squareform

---
### Step 2. Preparing a STAI Score Vector as an DIM's Input

I will extract the STAI scores of 119 young samples as a 1D array, which will be used as one of the inputs for DIM. To do this:

1. Using `Meta.csv`, I will filter the 119 participants based on the same three criteria used by the authors: age, availability of MRI data, and past/present psychiatric diagnosis. The metadata of the filtered participants will be saved as `Hyouth`.       

2. I will then extract the STAI summary scores of the participants in `Hyouth` as `stai_values`.

> **[Caution]** The participant IDs in `Meta.csv` are listed randomly, without ascending or descending order. The brain morphology data `yh_prob_5p_L` is sorted in ascending order of IDs. To align the participant order in `Hyouth` with `yh_prob_5p_L`, you must use `sort_values` to sort IDs in ascending order. Failing to do so may misalign the STAI distance matrix with the brain distance matrix, preventing meaningful relationships from being captured.

In [2]:
### Load the Data
meta = pd.read_csv('../data-from-authors/Meta.csv')
STAI = pd.read_csv('../data-from-authors/STAI_G_X2.csv')

### Filtering 'Young' Group Samples
## (1) filtering by age
youth = meta[meta['Age'].isin(['20-25', '25-30', '30-35'])]

## (2) filtering by availability of MRI data
exclude_subjects = ['sub-032339', 'sub-032341', 'sub-032459', 'sub-032370', 
                    'sub-032466', 'sub-032438', 'sub-032509']
youth = youth[~youth['Unnamed: 0'].isin(exclude_subjects)]

## (3) filtering by SKID Diagnoses
youth['SKID_Diagnoses'] = pd.Categorical(youth['SKID_Diagnoses'])
youth['SKID_Diagnoses_numeric'] = youth['SKID_Diagnoses'].cat.codes + 1 
Hyouth = youth[(youth['SKID_Diagnoses_numeric'] == 0) | (youth['SKID_Diagnoses_numeric'] == 10)]

### Leaving Only Relevant Info
columns_to_drop = list(range(3, 14)) + list(range(15, 21))  
Hyouth = Hyouth.drop(Hyouth.columns[columns_to_drop], axis=1)

### Merging STAI data with Hyouth
Hyouth = pd.merge(Hyouth, STAI, on = 'Unnamed: 0')

# IMPORTANT: Matching the order of subjects with brain morphology data
Hyouth = Hyouth.sort_values('Unnamed: 0').reset_index(drop = True)

### Constructing the Young Samples' STAI Scores Array
stai_values = Hyouth[['STAI_Trait_Anxiety']].to_numpy()

---
### Step 3. Constructing a Brain Morphology Distance Matrix as DIM's Input

Unlike the STAI scores for the test domain, the Brain Morphology data for the target domain in DIM must be provided as a distance matrix. To do this, we will use `yh_prob_5p_L.csv`, which was preprocessed with the 5% slice-level thresholding provided by the authors, to compute a 119×119 distance matrix.      

Since this data is in `(n_features, n_subjects)` format, we first need to transpose it before calculating the distance matrix. We will use `pdist` and `squareform` from `scipy.spatial.distance` for this purpose. This performs the same function as the line `HAnnaKBrain = dist(t(yh_prob_5p_L))` in `02_reproduction-with-authors-code.ipynb`.


In [3]:
# Load the tractography data
yh_prob_5p_L = pd.read_csv('../data-from-authors/tractfiles/yh_probmap_5p_L.csv')
yh_prob_5p_L = yh_prob_5p_L.iloc[:, 1:]  # Remove the first column (index)

# Compute the distance matrix
HAnnaKBrain = squareform(pdist(yh_prob_5p_L.T, metric = 'euclidean'))

---
### Step 4. Anna Karenina Model Testing with DIM's Level1 Class
Using `stai_values` and `HAnnaKBrain` prepared in Steps 2 and 3 as the main inputs for the `Level1` class, I will conduct Anna Karenina Model testing and examine whether the statistics can be reproduced. This process proceeds in three main stages:

1. **Model specification:** Set the parameters required by the `Level1` model, such as `model`, `distance`, `weighting`, and `dependency` (see *Overview of DIM Package*).      

2. **Model fitting:** Use the `fit` method of the `Level1` class to fit the model with `stai_values` and `HAnnaKBrain` as inputs. During this process, `Level1` internally generates an Anna-K distance matrix of the same 119×119 size as `HAnnaKBrain` from `stai_values`.       

3. **Permutation test:** Use the `permutation_test` method of the `Level1` class to assess the statistical significance of the correlation between the two distance matrices. The authors’ R-based `vegan::mantel` function uses one-sided testing (`greater`) for P-value calculation (see [here](https://vegandevs.github.io/vegan/reference/mantel.html)). Accordingly, we also use one-sided testing rather than a two-sided test.  

> **[NOTE]** Unlike `vegan::mantel`, DIM allows the researcher to flexibly choose between one-sided and two-sided statistical testing, which is a notable advantage.


In [6]:
### Defining the Model 
AnnaK_model = Level1(
    model = "AnnaK",
    distance = "Mean",
    weighting = "None",
    dependency = "spearman_r",
    normalize = False # not normalizing the stai_values as in the original study
)

### Fitting the Model
AnnaK_model.fit(stai_values, HAnnaKBrain)

### Conducting Permutation Test for Calculating P-value
mantel_result = AnnaK_model.permutation_test_discovery(
    n_perms = 10000,
    verbose = True,
    return_null_dist = True,
    seed = 42
)

mantel_result

Validation successful!


Permutation test (refitting): 100%|██████████| 10000/10000 [00:13<00:00, 714.73perm/s]


{'observed_stat': 0.1482987105846405,
 'p_value': np.float64(0.0426),
 'n_perms': 10000,
 'null_mean': np.float64(-0.0014572440253728474),
 'null_std': np.float64(0.08488472827755539),
 'computation_time': 14.028948068618774,
 'null_distribution': array([-0.05733099, -0.02783272,  0.15806741, ...,  0.04909881,
        -0.06928889,  0.04331321], shape=(10000,))}

---
### RESULTS

For the correlation between the Trait Anxiety distance matrix and the Brain Morphology distance matrix, the original R code provided by the authors and my custom Python package produced the following statistics (All values are truncated to three decimal places): 

- Authors’ code: *ρ* = 0.148, *P* = 0.041  
- Custom Python package: *ρ* = 0.148, *P* = 0.042  

Based on this, I consider the authors’ findings to have been accurately reproduced. Although there is a slight difference of approximately 0.001 in the *P*-value, this can be attributed to the inherent randomness of the permutation test. Even though the same seed (‘42’) was used, the internal random number generation algorithms of R and Python differ. Therefore, in our Preregistration Report, we pre-defined reproduction success/failure based on two criteria:  

1. Whether the estimated ρ is identical to the third decimal place.       
2. Whether a significant relationship (*P* < 0.05) is observed as in the original paper.       

Using these a-priori criteria, we conclude that the reproduction was **successful.**
